In [ ]:
%%time
!pip3 install -U sagemaker
!pip3 install -U boto3

#then restart kernel

In [ ]:
%%time
import os
import json

import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
import torch
from sagemaker.utils import unique_name_from_base

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()
prefix = 'transsion-poc-zhuan'
output_path = f"s3://{bucket}/{prefix}"

print("torch.__version__:{}".format(torch.__version__))
print("boto3.__version__:{}".format(boto3.__version__))
print("sagemaker.__version__:{}".format(sagemaker.__version__))
print("bucket:{}".format(bucket))
print("role:{}".format(role))

In [ ]:
%%time
!git clone https://github.com/wenet-e2e/wenet.git

!cp wenet/examples/librispeech/s0/*.sh wenet
!cp -r wenet/examples/librispeech/s0/local wenet
!mv -f train.py wenet/wenet/bin/train.py
!rm -fr wenet/examples
#!aws s3 cp --recursive s3://<>/<>/data wenet/data
!aws s3 cp --recursive s3://sagemaker-us-east-1-432088571089/dataset-transsion/data wenet/data
!aws s3 cp --recursive s3://sagemaker-us-east-1-432088571089/dataset-transsion/exp wenet/exp


In [ ]:
# creating a variable and storing the text
# that we want to search
search_text = "stage=0"
  
# creating a variable and storing the text
# that we want to add
replace_text = "stage=4"
  
# Opening our text file in read only
# mode using the open() function
with open(r'wenet/run.sh', 'r') as file:
  
    # Reading the content of the file
    # using the read() function and storing
    # them in a new variable
    data = file.read()
  
    # Searching and replacing the text
    # using the replace() function
    data = data.replace(search_text, replace_text)
  
# Opening our text file in write only
# mode to write the replaced content
with open(r'wenet/run.sh', 'w') as file:
  
    # Writing the replaced data in our
    # text file
    file.write(data)
  
# Printing Text replaced
print("Text replaced")

In [ ]:
# creating a variable and storing the text
# that we want to search
search_text = "stop_stage=5"
  
# creating a variable and storing the text
# that we want to add
replace_text = "stop_stage=4"
  
# Opening our text file in read only
# mode using the open() function
with open(r'wenet/run.sh', 'r') as file:
  
    # Reading the content of the file
    # using the read() function and storing
    # them in a new variable
    data = file.read()
  
    # Searching and replacing the text
    # using the replace() function
    data = data.replace(search_text, replace_text)
  
# Opening our text file in write only
# mode to write the replaced content
with open(r'wenet/run.sh', 'w') as file:
  
    # Writing the replaced data in our
    # text file
    file.write(data)
  
# Printing Text replaced
print("Text replaced")

In [ ]:
# manual modify all data.list in /wenet/data directory by replacing the path
# for example
#replace /export/data/en-asr-data/OpenSLR/LibriSpeech with /opt/ml/input/training
#PAY attentation on the data.list in train_960 directory, it's too large to process online, it can be downloaded to your pc,then processed & uploaded.

In [ ]:
%%time
instance_type = "ml.p3.2xlarge"

est = PyTorch(
    entry_point="run.sh",
    source_dir="./wenet",
    framework_version="1.11.0",
    py_version="py38",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size=10,
    disable_profiler=True,
    debugger_hook_config=False,
    base_job_name=prefix,
)


In [ ]:
from sagemaker.inputs import TrainingInput
prefix_dataset = "dataset-transsion/export"
loc =f"s3://{bucket}/{prefix_dataset}"

training = TrainingInput(
    s3_data_type='S3Prefix', # Available Options: S3Prefix | ManifestFile | AugmentedManifestFile
    s3_data=loc,
    distribution='FullyReplicated', # Available Options: FullyReplicated | ShardedByS3Key 
    input_mode='FastFile'
)

In [ ]:
%%time
job_name = est.fit({"training":training})
#job_name = est.fit()

In [ ]:
model_data = est.model_data
print("Model artifact saved at:\n", model_data)